# Simplified Model
This is an attempt to simplify the process considerably with the goal of creating a RESTful API for this.

In [2]:
df, images = get_images(csv_path='./annotations/train.csv', clahe=False)
data = df[(df.r<10)]

In [9]:
import scipy.misc

def datagen(batch_size = 32):
    while True:
        X = []
        Y = []
        for i, row in data.sample(n=batch_size).iterrows():
            img = images[row['source']]
            gt_center = row['x'], row['y']
            prop_center = (round(row['x']) + np.random.randint(-3, 4), round(row['y']) + np.random.randint(-3, 4))
            gt_radius = row['r']
            scale = 32
            extr, label = create_labeled_pair(img, gt_center, prop_center, gt_radius, scale)
            extr = extr[:, :, 0:1].astype(float)
            label = (label[0] + 15, label[1] + 15, label[2]*32)
            #extr -= extr.mean()
            #extr /= extr.std()
            X.append(np.expand_dims(extr, axis=0))
            Y.append(np.expand_dims(label, axis=0))
        X = np.concatenate(X)
        Y = np.concatenate(Y)
        yield X
        
gen = datagen()


i=-1
for _ in range(10):
    crats = next(gen)
    for crat in crats:
        i += 1
        scipy.misc.imsave(f'/home/alliedtoasters/craterfind/data/{i}.png', crat[:, :, 0])

/media/alliedtoasters/ExtraDrive1/miniconda3/envs/craters/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [6]:
scipy.misc.imsave('test_crater.jpg', out[0][:, :, 0])

/media/alliedtoasters/ExtraDrive1/miniconda3/envs/craters/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """Entry point for launching an IPython kernel.


In [36]:
import os
import json
import requests
import numpy as np
import subprocess

arr = np.random.rand(1024)

body = {
    "instances": [arr.tolist()]
}

body = json.dumps(body)

myurl = 'http://localhost:8501/v1/models/craters:predict'

batcmd=f"curl -d '{body}' -X POST {myurl}"
#print(batcmd, '\t')
print(subprocess.check_output(batcmd, shell=True).decode())

{
    "predictions": [[7.44050503, 7.73400879, 4.39179]
    ]
}


In [1]:
import pandas as pd
import numpy as np
from processing import get_images, create_labeled_example, create_labeled_pair
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

ModuleNotFoundError: No module named 'processing'

In [3]:
df, images = get_images(clahe=False)
data = df[(df.r<10)]

In [4]:
np.random.randint(-3, 4)

-3

In [5]:
def raw_datagen():
    while True:
        for i, row in data.sample(n=1).iterrows():
            img = images[row['source']]
            gt_center = row['x'], row['y']
            prop_center = (round(row['x']) + np.random.randint(-3, 4), round(row['y']) + np.random.randint(-3, 4))
            gt_radius = row['r']
            scale = 32
            extr, label = create_labeled_pair(img, gt_center, prop_center, gt_radius, scale)
            extr = extr[:, :, 0:1].astype(float)
            label = (label[0] + 15, label[1] + 15, label[2]*32)
        yield extr.flatten().tolist(), label

In [6]:
def datagen(batch_size = 32):
    while True:
        X = []
        Y = []
        for i, row in data.sample(n=batch_size).iterrows():
            img = images[row['source']]
            gt_center = row['x'], row['y']
            prop_center = (round(row['x']) + np.random.randint(-3, 4), round(row['y']) + np.random.randint(-3, 4))
            gt_radius = row['r']
            scale = 32
            extr, label = create_labeled_pair(img, gt_center, prop_center, gt_radius, scale)
            extr = extr[:, :, 0:1].astype(float)
            label = (label[0] + 15, label[1] + 15, label[2]*32)
            extr -= extr.mean()
            extr /= extr.std()
            X.append(np.expand_dims(extr, axis=0))
            Y.append(np.expand_dims(label, axis=0))
        X = np.concatenate(X)
        Y = np.concatenate(Y)
        yield X.reshape(batch_size, -1), Y.reshape(batch_size, -1)

In [7]:
gen = datagen()
x, y = next(gen)

In [8]:
model = Sequential()
model.add(Dense(1024, input_shape=(1024,), activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(3))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Use tf.cast instead.


In [10]:
model.fit_generator(
    gen,
    steps_per_epoch=100,
    epochs=30
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 4s 40ms/step - loss: 6.5012
Epoch 2/30
100/100 [==============================] - 3s 35ms/step - loss: 1.2542
Epoch 3/30
100/100 [==============================] - 3s 34ms/step - loss: 1.2933
Epoch 4/30
100/100 [==============================] - 4s 35ms/step - loss: 1.4136
Epoch 5/30
100/100 [==============================] - 4s 40ms/step - loss: 1.5453
Epoch 6/30
100/100 [==============================] - 4s 42ms/step - loss: 1.3475
Epoch 7/30
100/100 [==============================] - 4s 41ms/step - loss: 1.3120
Epoch 8/30
100/100 [==============================] - 4s 41ms/step - loss: 1.3130
Epoch 9/30
100/100 [==============================] - 4s 37ms/step - loss: 1.3628
Epoch 10/30
100/100 [==============================] - 4s 35ms/step - loss: 1.3081
Epoch 11/30
100/100 [==============================] - 3s 32ms/step - loss: 1.2703
Epoch 12/30
100/100 [===========================

In [11]:
model.save('./models/simple1.h2')

In [39]:
class ModelCaller(object):
    """Processes input and calls model."""
    
    def __init__(self, model):
        self.model = model
        
    def preprocess(self, inp):
        x = np.array(inp)
        x -= x.mean()
        x /= x.std()
        return x.reshape(1, -1)
    
    def __call__(self, inp):
        x = self.preprocess(inp)
        res = self.model.predict(x)
        return res
    
md = ModelCaller(model=model)

In [40]:
rgen = raw_datagen()

In [41]:
x, y = next(rgen)

In [42]:
md(x)

array([[14.472137, 16.155355,  9.416405]], dtype=float32)

(14.573405791210519, 15.557427409507, 8.855615343217057)